In [1]:
import numpy as np
import csv
from itertools import product
import itertools
import tensorflow.compat.v1 as tf 
tf.compat.v1.disable_eager_execution()
import sys, os
sys.path.append("../")
import copy
import pandas as pd
from tensorflow.python.platform import flags
from scipy.optimize import basinhopping
import time
from adf_data.census import census_data
from adf_data.credit import credit_data
from adf_data.compas import compas_data
from adf_data.default import default_data
from adf_data.bank import bank_data
from adf_data.heart import heart_data
from adf_model.tutorial_models import dnn
from adf_utils.utils_tf import model_prediction, model_argmax , layer_out
from adf_utils.config import census, credit, bank, compas, default, heart
from adf_tutorial.utils import cluster, gradient_graph
#from IPython.display import clear_output



FLAGS = flags.FLAGS

# step size of perturbation
perturbation_size = 1

def check_for_error_condition(conf, sess, x, preds, t, sens_params, input_shape, epsillon):
    """
    Check whether the test case is an individual discriminatory instance
    :param conf: the configuration of dataset
    :param sess: TF session
    :param x: input placeholder
    :param preds: the model's symbolic output
    :param t: test case
    :param sens: the index of sensitive feature
    :return: whether it is an individual discriminatory instance
    """
    global time_first_disc
    global start_time
    t = [t.astype('int')]    
    samples = m_instance( np.array(t), sens_params, conf )   
    pred = pred_prob(sess, x, preds, samples , input_shape )
    partition = clustering(pred,samples, sens_params , epsillon)
    if time_first_disc == 0:
        if (0 in model_argmax(sess, x, preds, samples.reshape(len(samples),input_shape[1]))) and \
           (1 in model_argmax(sess, x, preds, samples.reshape(len(samples),input_shape[1]))):
            time_first_disc =round( (time.time() - start_time) ,4)

    return  max(list(partition.keys())[1:]) - min(list(partition.keys())[1:]) , len(partition)-1#(len(partition) -1),
    
def seed_test_input(clusters, limit):
    """
    Select the seed inputs for fairness testing
    :param clusters: the results of K-means clustering
    :param limit: the size of seed inputs wanted
    :return: a sequence of seed inputs
    """
    i = 0
    rows = []
    max_size = max([len(c[0]) for c in clusters])
    while i < max_size:
        if len(rows) == limit:
            break
        for c in clusters:
            if i >= len(c[0]):
                continue
            row = c[0][i]
            rows.append(row)
            if len(rows) == limit:
                break
        i += 1
    return np.array(rows)

def clip(input, conf):
    """
    Clip the generating instance with each feature to make sure it is valid
    :param input: generating instance
    :param conf: the configuration of dataset
    :return: a valid generating instance
    """
    for i in range(len(input)):
        input[i] = max(input[i], conf.input_bounds[i][0])
        input[i] = min(input[i], conf.input_bounds[i][1])
    return input

class Local_Perturbation(object):
    """
    The  implementation of local perturbation
    """

    def __init__(self, sess, grad, x, n_values, sens_params, input_shape, conf):
        """
        Initial function of local perturbation
        :param sess: TF session
        :param grad: the gradient graph
        :param x: input placeholder
        :param n_value: the discriminatory value of sensitive feature
        :param sens_param: the index of sensitive feature
        :param input_shape: the shape of dataset
        :param conf: the configuration of dataset
        """
        self.sess = sess
        self.grad = grad
        self.x = x
        self.n_values = n_values
        self.input_shape = input_shape
        self.sens = sens_params
        self.conf = conf

    def __call__(self, x):
        """
        Local perturbation
        :param x: input instance for local perturbation
        :return: new potential individual discriminatory instance
        """

        # perturbation
        s = np.random.choice([1.0, -1.0]) * perturbation_size

        n_x = x.copy()
        for i in range(len(self.sens)):
            n_x[self.sens[i] - 1] = self.n_values[i]
            
        # compute the gradients of an individual discriminatory instance pairs
        ind_grad = self.sess.run(self.grad, feed_dict={self.x:np.array([x])})
        n_ind_grad = self.sess.run(self.grad, feed_dict={self.x:np.array([n_x])})

        if np.zeros(self.input_shape).tolist() == ind_grad[0].tolist() and \
           np.zeros(self.input_shape).tolist() == n_ind_grad[0].tolist():
            probs = 1.0 / (self.input_shape) * np.ones(self.input_shape)

            for sens in self.sens :
                probs[sens - 1] = 0              

        else:
            # nomalize the reciprocal of gradients (prefer the low impactful feature)
            grad_sum = 1.0 / (abs(ind_grad[0]) + abs(n_ind_grad[0]))

            for sens in self.sens :
                grad_sum[ sens - 1 ] = 0

            probs = grad_sum / np.sum(grad_sum)
        probs = probs/probs.sum()
        if True in np.isnan(probs):
            probs = 1.0 / (self.input_shape) * np.ones(self.input_shape)

            for sens in self.sens :
                probs[sens - 1] = 0
            probs = probs/probs.sum()


        # randomly choose the feature for local perturbation
        index = np.random.choice(range(self.input_shape) , p=probs)
        local_cal_grad = np.zeros(self.input_shape)
        local_cal_grad[index] = 1.0
        x = clip(x + s * local_cal_grad, self.conf).astype("int")
        return x
                
#--------------------------------------
def m_instance( sample, sens_params, conf):
    index = []
    m_sample = []
    for sens in sens_params:
        index.append([i for i in range(conf.input_bounds[sens-1][0], conf.input_bounds[sens-1][1]+1)])
      
    for ind in list(product(*index)):     
        temp = sample.copy()
        for i in range(len(sens_params)):
            temp[0][sens_params[i]-1] = ind[i]
        m_sample.append(temp)
    return np.array(m_sample)

def global_sample_select(clus_dic, sens_params):
    leng = 0
    for key in clus_dic.keys():
        if key == 'Seed':
            continue
        if len(clus_dic[key]) > leng:
            leng = len(clus_dic[key])
            largest = key
    
    sample_ind = np.random.randint(len(clus_dic[largest]))
    n_sample_ind = np.random.randint(len(clus_dic[largest]))
    
    sample = clus_dic['Seed']
    for i in range(len(sens_params)):
        sample[sens_params[i] -1] = clus_dic[largest][sample_ind][i]
    # returns one sample of largest partition and its pair
    return np.array([sample]),clus_dic[largest][n_sample_ind]


def local_sample_select(clus_dic, sens_params):
      
    k_1 = min(list(clus_dic.keys())[1:])
    k_2 = max(list(clus_dic.keys())[1:])
    
    sample_ind = np.random.randint(len(clus_dic[k_1]))
    n_sample_ind = np.random.randint(len(clus_dic[k_2]))

    sample = clus_dic['Seed']
    for i in range(len(sens_params)):
        sample[sens_params[i] -1] = clus_dic[k_1][sample_ind][i]
    return np.array([sample]),clus_dic[k_2][n_sample_ind]
    
def clustering(probs,m_sample, sens_params, epsillon):
    cluster_dic = {}
    cluster_dic['Seed'] = m_sample[0][0]
  
    for i in range(len(probs)):     
        #  to avoid k = Max + 1
        if probs[i] == 1.0:
            if (int( probs[i] / epsillon ) - 1) not in cluster_dic.keys():            
                cluster_dic[ (int( probs[i] / epsillon ) -1)] = [ [m_sample[i][0][j - 1] for j in sens_params]]           
            else:
                cluster_dic[ (int( probs[i] / epsillon ) -1)].append( [m_sample[i][0][j - 1] for j in sens_params] )              
        elif int( probs[i] / epsillon ) not in cluster_dic.keys():
                cluster_dic[ int( probs[i] / epsillon )] = [ [m_sample[i][0][j - 1] for j in sens_params] ]           
        else:
                cluster_dic[ int( probs[i] / epsillon)].append( [m_sample[i][0][j - 1] for j in sens_params] )

    return cluster_dic  

    
def pred_prob(sess, x, preds, m_sample, input_shape):
        probs = model_prediction(sess, x, preds, np.array(m_sample).reshape(len(m_sample),
                                                                input_shape[1]))[:,1:2].reshape(len(m_sample))
        return probs
#-------------------------------------------
    
def dnn_fair_testing(dataset, sens_params, model_path, cluster_num, 
                     max_global, max_local, max_iter, epsillon, max_k_iter):
    """
    
    The implementation of ADF
    :param dataset: the name of testing dataset
    :param sensitive_param: the index of sensitive feature
    :param model_path: the path of testing model
    :param cluster_num: the number of clusters to form as well as the number of
            centroids to generate
    :param max_global: the maximum number of samples for global search
    :param max_local: the maximum number of samples for local search
    :param max_iter: the maximum iteration of global perturbation
    """
    data = {"census":census_data, "credit":credit_data, "bank":bank_data, "compas":compas_data, 
            "default": default_data, "heart":heart_data}
    data_config = {"census":census, "credit":credit, "bank":bank, "compas":compas, "default":default,
                  "heart":heart}
    # prepare the testing data and model
    X, Y, input_shape, nb_classes = data[dataset]()
    tf.set_random_seed(1234)

    config = tf.ConfigProto(device_count = {'GPU': 0})
    config.allow_soft_placement= True

    sess = tf.Session(config=config)
    x = tf.placeholder(tf.float32, shape=input_shape)
    y = tf.placeholder(tf.float32, shape=(None, nb_classes))
    model = dnn(input_shape, nb_classes)   

    preds = model(x)
    saver = tf.train.Saver()
    model_path = model_path + dataset + "/test.model"
    saver.restore(sess, model_path)

    # construct the gradient graph
    grad_0 = gradient_graph(x, preds)

    # build the clustering model
    clf = cluster(dataset, cluster_num)
    clusters = [np.where(clf.labels_ == i) for i in range(cluster_num)]


    local_sam = np.load('../results/census/RQ1&2/10_1000/local_samples_0.npy').astype('int32')
    global_sam = np.load('../results/census/RQ1&2/10_1000/global_samples_0.npy').astype('int32')
    temp_g = np.zeros((1,input_shape[1]))
    
    with open('../results/census/RQ1&2/10_1000/global_samples_0_test.csv', 'w') as f:
        writer = csv.writer(f)
        for ind in range(len(global_sam)):
            m_sample = m_instance( np.array([global_sam[ind][:-6]]) , sens_params, data_config[dataset] ) 
            writer.writerows(m_sample.reshape((len(m_sample),input_shape[1])))

    with open('../results/census/RQ1&2/10_1000/local_samples_0_test.csv', 'w') as f:
        writer = csv.writer(f)
        
        for ind in range(len(local_sam)):
            m_sample = m_instance( np.array([local_sam[ind][:-6]]) , sens_params, data_config[dataset] ) 
            writer.writerows(m_sample.reshape((len(m_sample),input_shape[1])))
    df_l = pd.read_csv('../results/census/RQ1&2/10_1000/local_samples_0_test.csv',header=None)  
    df_g = pd.read_csv('../results/census/RQ1&2/10_1000/global_samples_0_test.csv',header=None)
    df_g['label'] = model_argmax(sess, x, preds, df_g.to_numpy())
    df_l['label'] = model_argmax(sess, x, preds, df_l.to_numpy())
    
    g = pd.pivot_table(df_g, values="label", index=list(np.setxor1d(df_g.columns[:-1] ,
                                                                          np.array(sens_params)-1)), aggfunc=np.sum)
    l = pd.pivot_table(df_l, values="label", index=list(np.setxor1d(df_g.columns[:-1] ,
                                                                       np.array(sens_params)-1)), aggfunc=np.sum)
    g_dis = (len(m_sample) - g.loc[(g['label']!= 0) & (g['label'] != len(m_sample))]['label'].to_numpy()).sum()
    l_dis = (len(m_sample) - l.loc[(l['label']!= 0) & (l['label'] != len(m_sample))]['label'].to_numpy()).sum()

    adf_l = 0
    adf_g = 0
    for sens in sens_params:
        temp_g = pd.pivot_table(df_g, values="label", index=list(np.setxor1d(df_g.columns[:-1] ,
                                                                           [sens-1])), aggfunc=np.sum)
        temp_l = pd.pivot_table(df_l, values="label", index=list(np.setxor1d(df_g.columns[:-1] ,
                                                            [sens-1])), aggfunc=np.sum)
        inp_bound = data_config[dataset].input_bounds[sens -1][1] - data_config[dataset].input_bounds[sens -1 ][0]
        adf_g += len(temp_g.loc[(temp_g['label']> 0) & (temp_g['label'] < (inp_bound + 1))])
        adf_l += len(temp_l.loc[(temp_l['label']> 0) & (temp_l['label'] < (inp_bound + 1))])

    print('time',time.time() - time1)
    print('Global disc =', g_dis)
    print('local disc  =',  l_dis)
    print('Total loc samples  = ', len(local_sam)) 
    print('Total glob samples = ', len(global_sam)) 
    print('Total Dis = ',g_dis + l_dis)
    print('Total Dis ADF  = ',adf_g + adf_l)
    print('Total Dis ADF G = ',adf_g)
    print('Total Dis ADF L = ',adf_l)

    print('Local search success rate = ',(l_dis  / (len(local_sam) * \
                                                         len(m_sample)) * 100), '%')
    print('Global search success rate = ',(g_dis / (len(global_sam) * \
                                                         len(m_sample)) * 100), '%')



    
#------------------------
def main(argv=None):
    dnn_fair_testing(dataset = FLAGS.dataset, 
                     sens_params = FLAGS.sens_params,
                     model_path  = FLAGS.model_path,
                     cluster_num = FLAGS.cluster_num,
                     max_global  = FLAGS.max_global,
                     max_local   = FLAGS.max_local,
                     max_iter    = FLAGS.max_iter,
                     epsillon    = FLAGS.epsillon,
                     max_k_iter  = FLAGS.max_k_iter)

if __name__ == '__main__':
    flags.DEFINE_string("dataset", "census", "the name of dataset")
    flags.DEFINE_string('model_path', '../models/', 'the path for testing model')
    flags.DEFINE_integer('cluster_num', 4, 'the number of clusters to form as well as the number of centroids to generate')
    flags.DEFINE_integer('max_global', 1000, 'maximum number of samples for global search')#1000
    flags.DEFINE_integer('max_local', 100, 'maximum number of samples for local search')#1000
    flags.DEFINE_integer('max_iter', 10, 'maximum iteration of global perturbation')
    flags.DEFINE_list('sens_params', [9,8,1], 'sensitive parameters index.1 for age, 9 for gender, 8 for race')
    flags.DEFINE_float('epsillon', 0.025, 'the value of epsillon for partitioning')
    flags.DEFINE_float('max_k_iter', 0.3, 'the percentage of max_iter if K is not increased')
    tf.app.run()


INFO:tensorflow:Restoring parameters from ../models/census/test.model


I0801 15:51:57.285918 139940864554816 saver.py:1399] Restoring parameters from ../models/census/test.model


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



W0801 15:51:57.325730 139940864554816 deprecation.py:341] From /usr/lib/python3/dist-packages/tensorflow/python/util/dispatch.py:1096: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



time 20.16620969772339
Global disc = 1222
local disc  = 1025600
Total loc samples  =  19852
Total glob samples =  41
Total Dis =  1026822
Total Dis ADF  =  421197
Total Dis ADF G =  193
Total Dis ADF L =  421004
Local search success rate =  57.40255669733808 %
Global search success rate =  33.11653116531165 %


SystemExit: 

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
df_g

NameError: name 'df_g' is not defined

In [6]:

sum_g=0
time1 =time.time()
sens = np.array([9,8,1])
x=pd.pivot_table(df_g, values="label", index=list(np.setxor1d(df_g.columns[:-1] ,
                                                       sens-1)), aggfunc=np.sum)

x.loc[(x['label']> 0) & (x['label']< 9)]


label
1 2 3 4 5  6 9 10 11 12        
0 6 5 0 1  1 1 0  40 38       3
    6 0 1  2 0 0  40 100      6
1 5 6 0 1  2 2 0  39 39       3
2 4 8 0 3  4 3 0  37 39       3
4 5 1 4 9  2 4 2  43 3        1
  6 1 4 9  2 4 2  43 3        8
5 6 0 4 10 4 3 2  42 2        2
  7 2 5 10 3 4 3  44 4        3
6 9 1 0 13 1 1 1  59 10       2
7 9 1 0 13 2 1 1  59 10       1

In [5]:
x


IndentationError: unexpected indent (<ipython-input-5-25b9086a5f54>, line 2)

In [39]:
151034+126119+127304

404457